In [ ]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns


from scipy import signal
from scipy.fft import fft,fftfreq
from scipy.fft import fftshift

import tensorflow as tf
import keras

from keras.models import Sequential,load_model
from keras.layers import Dense, Conv2D ,LSTM, MaxPooling2D , Flatten , Dropout , BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# konverter windowing

def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        
        features = df.iloc[i : (i + window), feature_col_number].values
        target = df.iloc[(i + window), target_col_number]
     
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).astype(np.float64).reshape(-1, 1)

In [ ]:
gc_df = pd.read_csv('../input/deapdata/(S01)/Raw EEG Data/.csv format/S01G1AllRawChannels.csv')
gc_df.drop(['Unnamed: 14'], axis=1, inplace=True)
gc_df.head(10)

In [ ]:
gc_df.corr()

In [ ]:
window_size = 50 # 3 days
(X, y) = window_data(gc_df, window_size, 1, 1)

In [ ]:
y.shape,X.shape
#pd.DataFrame(y)


In [ ]:
#AF3	AF4	F3	F4	F7	F8	FC5	FC6	O1	O2	P7	P8	T7	T8
#window_size = 128
(AF3, _) = window_data(gc_df, window_size, 0, 0)
(AF4, _) = window_data(gc_df, window_size, 1, 1)
(F3, _) = window_data(gc_df, window_size,  2, 2)
(F4, _) = window_data(gc_df, window_size,  3, 3)
(F7, _) = window_data(gc_df, window_size,  4, 4)
(F8, _) = window_data(gc_df, window_size,  5, 5)
(FC5, _) = window_data(gc_df, window_size, 6, 6)
(FC6, _) = window_data(gc_df, window_size, 7, 7)
(O1, _) = window_data(gc_df, window_size,  8, 8)
(O2, _) = window_data(gc_df, window_size,  9, 9)
(P7, _) = window_data(gc_df, window_size,  10, 10)
(P8, _) = window_data(gc_df, window_size,  11, 11)
(T7, _) = window_data(gc_df, window_size,  12, 12)
(T8, _) = window_data(gc_df, window_size,  13, 13)


In [ ]:
#F3

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler   = StandardScaler()
X = scaler.fit_transform(X)


AF3 = scaler.fit_transform(AF3)
AF4 = scaler.fit_transform(AF4)
F3 = scaler.fit_transform(F3)
F4 = scaler.fit_transform(F4)
F7 = scaler.fit_transform(F7)
F8 = scaler.fit_transform(F8)
FC5 = scaler.fit_transform(FC5)
FC6 = scaler.fit_transform(FC6)
O1 = scaler.fit_transform(O1)
O2 = scaler.fit_transform(O2)
P7 = scaler.fit_transform(P7)
P8 = scaler.fit_transform(P8)
T7 = scaler.fit_transform(T7)
T8 = scaler.fit_transform(T8)

In [ ]:
AF4.shape

In [ ]:
#AF3	AF4	F3	F4	F7	F8	FC5	FC6	O1	O2	P7	P8	T7	T8
split = int(0.8 * len(X))
AF3_train = AF3[: split - 1]
AF4_train = AF4[: split - 1]
F3_train = F3[: split - 1]
F4_train = F4[: split - 1]
F7_train = F7[: split - 1]
F8_train = F8[: split - 1]
FC5_train = FC5[: split - 1]
FC6_train = FC6[: split - 1]
O1_train  = O1[: split - 1]
O2_train  = O2[: split - 1]
P7_train  = P7[: split - 1]
P8_train  = P8[: split - 1]
T7_train  = T7[: split - 1]
T8_train  = T8[: split - 1]


AF3_test = AF3[split:]
AF4_test = AF4[split:]
F3_test =  F3[split:]
F4_test =  F4[split:]
F7_test =  F7[split:] 
F8_test =  F8[split:]
FC5_test = FC5[split:]
FC6_test = FC6[split:]
O1_test =  O1[split:]
O2_test =  O2[split:]
P7_test =  P7[split:]
P8_test =  P8[split:]
T7_test =  T7[split:]
T8_test =  T8[split:]


In [ ]:
#AF4_train.shape,X_train.shape

In [ ]:
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
#X_test  =  X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

AF3_train = AF3_train.reshape((AF3_train.shape[0], AF3_train.shape[1], 1))
AF4_train = AF4_train.reshape((AF4_train.shape[0], AF4_train.shape[1], 1))
F3_train  = F3_train.reshape((F3_train.shape[0], F3_train.shape[1], 1))
F4_train  = F4_train.reshape((F4_train.shape[0], F4_train.shape[1], 1))
F7_train  = F7_train.reshape((F7_train.shape[0], F7_train.shape[1], 1))
F8_train  = F8_train.reshape((F8_train.shape[0], F8_train.shape[1], 1))
FC5_train = FC5_train.reshape((FC5_train.shape[0], FC5_train.shape[1], 1))
FC6_train = FC6_train.reshape((FC6_train.shape[0], FC6_train.shape[1], 1))
O1_train  = O1_train.reshape((O1_train.shape[0], O1_train.shape[1], 1))
O2_train  = O2_train.reshape((O2_train.shape[0], O2_train.shape[1], 1))
P7_train  = P7_train.reshape((P7_train.shape[0], P7_train.shape[1], 1))
P8_train  = P8_train.reshape((P8_train.shape[0], P8_train.shape[1], 1))
T7_train  = T7_train.reshape((T7_train.shape[0], T7_train.shape[1], 1))
T8_train  = T8_train.reshape((T8_train.shape[0], T8_train.shape[1], 1))
#data_train = np.concatenate((X_train, DADemand_train), axis=2)
data_train = np.concatenate((AF3_train,	AF4_train,F3_train,F4_train,F7_train,F8_train
                             ,FC5_train,FC6_train,O1_train,O2_train,P7_train,
                             P8_train,T7_train,T8_train), axis=2)

AF3_test = AF3_test.reshape((AF3_test.shape[0], AF3_test.shape[1], 1))
AF4_test = AF4_test.reshape((AF4_test.shape[0], AF4_test.shape[1], 1))
F3_test  = F3_test.reshape((F3_test.shape[0], F3_test.shape[1], 1))
F4_test  = F4_test.reshape((F4_test.shape[0], F4_test.shape[1], 1))
F7_test  = F7_test.reshape((F7_test.shape[0], F7_test.shape[1], 1))
F8_test  = F8_test.reshape((F8_test.shape[0], F8_test.shape[1], 1))
FC5_test = FC5_test.reshape((FC5_test.shape[0], FC5_test.shape[1], 1))
FC6_test = FC6_test.reshape((FC6_test.shape[0], FC6_test.shape[1], 1))
O1_test  = O1_test.reshape((O1_test.shape[0], O1_test.shape[1], 1))
O2_test  = O2_test.reshape((O2_test.shape[0], O2_test.shape[1], 1))
P7_test  = P7_test.reshape((P7_test.shape[0], P7_test.shape[1], 1))
P8_test  = P8_test.reshape((P8_test.shape[0], P8_test.shape[1], 1))
T7_test  = T7_test.reshape((T7_test.shape[0], T7_test.shape[1], 1))
T8_test  = T8_test.reshape((T8_test.shape[0], T8_test.shape[1], 1))
#data_test = np.concatenate((X_test, db_test, dew_test, DADemand_test, DALMP_test, DAEC_test, DACC_test, DAMLC_test, RTLMP_test, RTEC_test, RTCC_test, RTMLC_test), axis=2)
data_test= np.concatenate((AF3_test,AF4_test,F3_test,F4_test,F7_test,F8_test
                             ,FC5_test,FC6_test,O1_test,O2_test,P7_test,
                             P8_test,T7_test,T8_test), axis=2)


#data_test = np.concatenate((X_test, DADemand_test), axis=2)

In [ ]:
data_train.shape,data_test.shape

In [ ]:

gc_df['AF3'].replace(0.0, np.nan, inplace=True)
gc_df['AF3'].fillna(method='ffill', inplace=True)
gc_df['AF3'][24:8760].plot()


In [ ]:
window_size = 50
(X, y) = window_data(gc_df, window_size, 0, 0)
y.shape,X.shape

In [ ]:
plt.plot(X[window_size])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
split = int(0.8 * len(X))
X_train = X[: split - 1]
y_train = y[: split - 1]
X_test = X[split:]
y_test = y[split:]
X_train.shape,X_test .shape,y_train.shape,y_test.shape

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_train.shape,X_test.shape

# LSTM basic 1 CHANEL

In [ ]:
def basic_LSTM(window_size=5, n_features=1):
    new_model = keras.Sequential()
    new_model.add(tf.keras.layers.LSTM(100, input_shape=(window_size, n_features), activation='relu'))
    new_model.add(tf.keras.layers.Flatten())
    new_model.add(tf.keras.layers.Dense(1500, activation='relu'))
    new_model.add(tf.keras.layers.Dense(100, activation='linear'))
    new_model.add(tf.keras.layers.Dense(1))
    new_model.compile(optimizer="adam", loss="mean_squared_error")    
    return new_model

In [ ]:
ls_model = basic_LSTM(window_size=window_size, n_features=X_train.shape[2])

In [ ]:
ls_model.summary()

In [ ]:
ls_history = ls_model.fit(X_train ,y_train, epochs=10, shuffle=False, batch_size=100, verbose=1)

In [ ]:
plt.plot(ls_history.history["loss"])
plt.title("loss_function - LSTM Basic")
plt.legend(["loss"])
plt.show()

In [ ]:
ls_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
from sklearn.metrics import r2_score
predictions = ls_model.predict(X_test)
test_r2 = r2_score(y_test, predictions)
test_r2

In [ ]:
lstm_acc_df = pd.DataFrame()
lstm_acc_df['Actual'] = y_test[:,0]
lstm_acc_df['Predict'] = predictions[:,0]
lstm_acc_df[48:240].plot()

# cnn baSIC 1 CHANEL

In [ ]:
def basic_conv1D(n_filters=10, fsize=5, window_size=5, n_features=2):
    new_model = keras.Sequential()
    # Hypothetically, we could also tune the padding and activation here.
    new_model.add(tf.keras.layers.Conv1D(n_filters, fsize, padding="same", activation="relu", input_shape=(window_size, n_features)))
    # Flatten will take our convolution filters and lay them out end to end so our dense layer can predict based on the outcomes of each
    new_model.add(tf.keras.layers.Flatten())
    new_model.add(tf.keras.layers.Dense(1800, activation='relu'))
    new_model.add(tf.keras.layers.Dense(100))
    new_model.add(tf.keras.layers.Dense(1))
    new_model.compile(optimizer="adam", loss="mean_squared_error")    
    return new_model


univar_model = basic_conv1D(n_filters=24, fsize=8, window_size=window_size, n_features=X_train.shape[2])
univar_model.summary()

In [ ]:
univar_hist = univar_model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=10, verbose=1)

In [ ]:
univar_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
from sklearn.metrics import r2_score
predictions = univar_model.predict(X_test)
test_r2 = r2_score(y_test, predictions)
test_r2

In [ ]:
plt.plot(univar_hist.history["loss"])
plt.title("loss_function - Conv1D Deep, w/ 1 variable")
plt.legend(["loss"])

plt.show()

In [ ]:
conv_acc_df = pd.DataFrame()
conv_acc_df['Actual'] = y_test[:,0]
conv_acc_df['Predict'] = predictions[:,0]
conv_acc_df.head(10)

In [ ]:
conv_acc_df[24:480].plot()

# lstm MULTY

In [ ]:
ls_model = basic_LSTM(window_size=window_size, n_features=data_train.shape[2])

In [ ]:
ls_model.summary()

In [ ]:
summa = np.sum(data_train)
check_nan = np.isnan(summa)
check_nan

In [ ]:
ls_history = ls_model.fit(data_train ,y_train, epochs=10, shuffle=False, batch_size=100, verbose=1)

In [ ]:
plt.plot(ls_history.history["loss"])
plt.title("loss_function - LSTM Multivariate")
plt.legend(["loss"])
plt.show()

In [ ]:

test = ls_model.evaluate(data_test, y_test, verbose=0)
if(not np.isnan(test)):
    print(test)
else:
    print("nan failure. :(")

In [ ]:

if(not np.isnan(test)):
    from sklearn.metrics import r2_score
    predictions = ls_model.predict(data_test)
    test_r2 = r2_score(y_test, predictions)
    print(test_r2)

In [ ]:
if(not np.isnan(test)):
    lstm_acc_df = pd.DataFrame()
    lstm_acc_df['Actual'] = y_test[:,0]
    lstm_acc_df['Predict'] = predictions[:,0]
    lstm_acc_df[24:480].plot()

# CNN 1D FULL CHANEL

In [ ]:
test_model = basic_conv1D(n_filters=25, fsize=8, window_size=window_size, n_features=data_train.shape[2])

In [ ]:
test_model.summary()

In [ ]:
train_history = test_model.fit(data_train, y_train, epochs=10, shuffle=False, batch_size=10, verbose=1)

In [ ]:
plt.plot(train_history.history["loss"])
plt.title("loss_function - Conv1D Deep, w/ 12 variable")
plt.legend(["loss"])

plt.show()

In [ ]:
test_model.evaluate(data_test, y_test, verbose=0)

In [ ]:
from sklearn.metrics import r2_score
predictions = test_model.predict(data_test)
test_r2 = r2_score(y_test, predictions)
test_r2

In [ ]:
conv_acc_df = pd.DataFrame()
conv_acc_df['Actual'] = y_test[:,0]
conv_acc_df['Predict'] = predictions[:,0]
conv_acc_df.head(10)

In [ ]:
conv_acc_df[24:480].plot()

# CNN 2D

In [ ]:
def basic_conv2D(n_filters=10, fsize=5, window_size=5, n_features=2):
    new_model = keras.Sequential()
    # Hypothetically, we could also tune the padding and activation here.
    new_model.add(tf.keras.layers.Conv2D(n_filters, (1,fsize), padding="same", activation="relu", input_shape=(window_size, n_features, 1)))
    # Flatten will take our convolution filters and lay them out end to end so our dense layer can predict based on the outcomes of each
    new_model.add(tf.keras.layers.Flatten())
    new_model.add(tf.keras.layers.Dense(1000, activation='relu'))
    new_model.add(tf.keras.layers.Dense(100))
    new_model.add(tf.keras.layers.Dense(1))
    new_model.compile(optimizer="adam", loss="mean_squared_error")    
    return new_model

In [ ]:
data_train_wide = data_train.reshape((data_train.shape[0], data_train.shape[1], data_train.shape[2], 1))
data_test_wide = data_test.reshape((data_test.shape[0], data_test.shape[1], data_test.shape[2], 1))

In [ ]:
data_train_wide.shape

In [ ]:
m2 = basic_conv2D(n_filters=24, fsize=2, window_size=window_size, n_features=data_train_wide.shape[2])

In [ ]:
m2.summary()

In [ ]:
m2_hist = m2.fit(data_train_wide, y_train, epochs=10)

In [ ]:
plt.plot(m2_hist.history["loss"])
plt.title("loss_function - Conv2D Deep, w/ 12 variable")
plt.legend(["loss"])
# 
plt.show()

In [ ]:
m2.evaluate(data_test_wide, y_test, verbose=0)

In [ ]:
from sklearn.metrics import r2_score
predictions = m2.predict(data_test_wide)
test_r2 = r2_score(y_test, predictions)
test_r2

In [ ]:
conv_acc_df = pd.DataFrame()
conv_acc_df['Actual'] = y_test[:,0]
conv_acc_df['Predict'] = predictions[:,0]
conv_acc_df.head(10)

In [ ]:
conv_acc_df[24:240].plot()

# CNN DEEP 2D

In [ ]:
def deeper_conv2D(n_filters=10, fsize=5, window_size=5, n_features=2, hour_filter=8):
    new_model = keras.Sequential()
    new_model.add(tf.keras.layers.Conv2D(n_filters, (1,fsize), padding="same", activation="linear", input_shape=(window_size, n_features, 1)))
    new_model.add(tf.keras.layers.Conv2D(n_filters, (hour_filter, 1), padding="same", activation="relu"))
    # Flatten will take our convolution filters and lay them out end to end so our dense layer can predict based on the outcomes of each
    new_model.add(tf.keras.layers.Flatten())
    new_model.add(tf.keras.layers.Dense(1000, activation='relu'))
    new_model.add(tf.keras.layers.Dense(100))
    new_model.add(tf.keras.layers.Dense(1))
    new_model.compile(optimizer="adam", loss="mean_squared_error")    
    return new_model

In [ ]:
m3 = deeper_conv2D(n_filters=24, fsize=2, window_size=window_size, n_features=data_train_wide.shape[2], hour_filter=8)

In [ ]:
m3.summary()

In [ ]:
m3_hist = m3.fit(data_train_wide, y_train, epochs=10)

In [ ]:
plt.plot(m3_hist.history["loss"])
plt.title("loss_function - Conv2D->conv2D, w/ 12 variable")
plt.legend(["loss"])
# 
plt.show()

In [ ]:
m3.evaluate(data_test_wide, y_test, verbose=0)

In [ ]:
from sklearn.metrics import r2_score
predictions = m3.predict(data_test_wide)
test_r2 = r2_score(y_test, predictions)
test_r2

In [ ]:
conv_acc_df = pd.DataFrame()
conv_acc_df['Actual'] = y_test[:,0]
conv_acc_df['Predict'] = predictions[:,0]
conv_acc_df.head(10)

In [ ]:
conv_acc_df[24:240].plot()

In [ ]:
# Dumb model
prev_val = y_test[0]
sse = 0

for n in range(0, len(y_test)-1):
    err = y_test[n] - prev_val
    sq_err = err ** 2
    sse = sse + sq_err
    prev_val = y_test[n]

mse = sse / n
mse

In [ ]:
predict = pd.DataFrame(y_test, columns = ["dumb_predict"])
predict = predict.shift(1)
predict["actual"] = y_test
predict.fillna(value=0, inplace=True)
correct_cols = ["actual", "dumb_predict"]
predict = predict.reindex(columns=correct_cols)
predict


In [ ]:
predict[24:96].plot()

In [ ]:
from sklearn.metrics import r2_score
predictions = predict["dumb_predict"].values
test_r2 = r2_score(y_test, predictions)
test_r2